# Charts

In [ ]:
from datascience import *
from cs104 import *
import numpy as np

%matplotlib inline

## 1. Table and visualization review

In [ ]:
# Load data from the last lecture and clean it up
greenland_climate = Table.read_table('data/climate_upernavik.csv')
greenland_climate = greenland_climate.relabeled('Precipitation (millimeters)', "Precip (mm)")
tidy_greenland = greenland_climate.where('Air temperature (C)', are.not_equal_to(999.99))
tidy_greenland = tidy_greenland.where('Sea level pressure (mbar)', are.not_equal_to(9999.9))
tidy_greenland.show(4)

### Think-Pair-Share

**Question:** For the year 1900, make a visualization of each month and the amount of precipitation. Does visualization confirm your intuition of what you should expect to see? Why or why not?  

*Hint:* Think carefully about what type of visualization you should use. 

In [ ]:
tidy_greenland.where('Year', are.equal_to(1900)).barh('Month', 'Precip (mm)')

**Question:** How about if we want the data in inches?

*Hint:* 25.4 millimeters are equal to one inch.

In [ ]:
precip_in_inches = tidy_greenland.with_column("Precip (in)",
                                             tidy_greenland.column("Precip (mm)") / 25.4)

precip_in_inches.where('Year', are.equal_to(1900)).barh('Month', 'Precip (in)')

### A Little Practice Fixing Programming Mistakes

For fun, let's look at only recent data between 1950 and 1995.  Here's our first attempt

In [ ]:
recent = tidy_greenland.where(Year, are.strictly_between("1950","1995")) #should throw an error

How do we debug this error message? 

In [ ]:
# Ok. Try to fix something 
recent = tidy_greenland.where("Year", are.strictly_between("1950", "1995")) #should throw an error

Always good to read the documentation for the functions and methods you are using.  Docs for [are.strictly_between](https://www.cs.williams.edu/~cs104/auto/python-library-ref.html#strictly_between).  What types of parameters does that method take take?

In [ ]:
#print out types and comment out pieces of code to debug 
type("1950") 
type(1950) #comment this line to see the previous line 

The fixed version:

In [ ]:
recent = tidy_greenland.where("Year", are.strictly_between(1950, 1995))
recent.show(3)

Great! No errors, but did we get the correct answer?  How could we sanity check?  Look for smallest and largest year left in the table.

In [ ]:
years_chosen = recent.column("Year")
print(min(years_chosen))
print(max(years_chosen))

That doesn't look right.  Back to the docs for [are.strictly_between](https://www.cs.williams.edu/~cs104/auto/python-library-ref.html#strictly_between)...

In [ ]:
recent = tidy_greenland.where("Year", are.between_or_equal_to(1950, 1995))
years_chosen = recent.column("Year")
print(min(years_chosen))
print(max(years_chosen))

That looks good.  Let's visualize the Februrary temperatures in the `recent` table.

### Plot customizations

In [ ]:
feb = recent.where('Month', are.equal_to(2))
plot = feb.scatter('Year', 'Air temperature (C)')

Let's "zoom in"!

In [ ]:
plot = feb.scatter('Year', 'Air temperature (C)')
plot.set_ylim(-15, -25)
plot.set_xlim(1970, 1990)

We can change the title and labels on the graph. 

In [ ]:
plot = feb.scatter('Year', 'Air temperature (C)')
plot.set_ylim(-15, -25)
plot.set_xlim(1970, 1990)
plot.set_title("February in Greenland")
plot.set_ylabel("Air temp in celcius")
plot.set_xlabel("Year of data")

## 2. Visualization Discussion

### Choosing betweeen visualizations

**Question:** What's wrong with this visualization? How could we fix it? 

In [ ]:
tidy_greenland.plot('Sea level pressure (mbar)', 'Precip (mm)')

A: Line plots should only be used when it's meaningful to connect points along the x-axis (e.g. time or distance). We can change this to a scatter plot to analyze the relationship between these two numerical variables. 

In [ ]:
tidy_greenland.scatter('Sea level pressure (mbar)', 'Precip (mm)')

### Think-pair-share

**Question:** Does a warmer July correlate with a warmer August?  Best we can do with our current table

In [ ]:
with Figure(1,2):
    tidy_greenland.where("Month", are.equal_to(7)).plot("Year", 'Air temperature (C)', title='July')
    tidy_greenland.where("Month", are.equal_to(8)).plot("Year", 'Air temperature (C)', title='Aug')

How can we improve on this?
* What table would be useful to have?
* How would you visualize the data in that table?

In [ ]:
temps_by_month = Table().read_table("data/temps_by_month_upernavik.csv")
temps_by_month.show(5)

In [ ]:
temps_by_month.select("Year", "Jul", "Aug").plot("Year")  # overlay two lines -- more on this below!

Sort of see a correlation, but does time actually matter here???  Pick a plot that focusses on the two variables we care about: July temps and August temps.

In [ ]:
temps_by_month.scatter("Jul", "Aug")

**Making it interactive.** We can't resist making an interactive visualization for looking at any two months in the same calendar year.  We'll look at the code in more detail in a week or two, so for now, just enjoy!

In [ ]:
def month_correlations(first, second):
    plot = temps_by_month.scatter(first, second)
    plot.set_title("Correlation between " + first + " and " + second + " Temps");
    
months = make_array("Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec")
interact(month_correlations, first=Choice(months), second=Choice(months))

Right now, we're looking at everything *qualitatively*. By the end of this class, we'll be able to *quantify* this relationship (if you're curious--we'll use correlation coefficients). 

## 3. Overlaid graphs

### Overlaid bar charts

In [ ]:
majors = Table.read_table("data/majors.csv")
div3 = majors.where("Division", are.equal_to(3)).drop("Division")
div3

In [ ]:
# First graph for 2008-2012
div3.barh("Major", "2008-2012")

In [ ]:
# Second graph from 2018-2021
div3.barh("Major", "2018-2021")

Overlaid graph puts the two graphs together to make comparison easier.

The package we're using will automatically make overlaid graphs with the remainder of the columns if you give it just one parameter. 

In [ ]:
div3.barh("Major")

### Overlaid line plots

In [ ]:
temps_by_month.show(5)

As with bar charts, if you supply only one parameter, the `plot` method will plot a line for every other column.

In [ ]:
temps_by_month.plot("Year")

Qualitatively, we can see that the plot above has too much information on it which makes it not very useful for understand trends. 

In [ ]:
temps_by_month.select("Year", "Feb", "Aug").plot("Year")

### Overlaid scatter plots 
* We want to plot points (the values of two numerical variables) from different groups on the same graph.
* A new approach.  Use categorical variable to break the rows into groups of related points in the plot.

In [ ]:
finch_1975 = Table().read_table("data/finch_beaks_1975.csv")
finch_1975.show(6)

In [ ]:
finch_1975.scatter("Beak length, mm", "Beak depth, mm", group="species")

**Takeaway:** The overlaid scatter plot above helps us very quickly discern differences between groups. In this case, we can quickly tell that the two Finch species have evolved (via natural selection) to have different beak characteristics. 

One last plot, this time with lines fitted to the data:

In [ ]:
finch_1975.scatter("Beak length, mm", "Beak depth, mm", group="species", fit_line=True)